In [314]:
import pandas as pd
import time
from datetime import datetime, timedelta
import matplotlib as plt
import numpy as np
import warnings

In [315]:
warnings.filterwarnings('ignore')

In [317]:
df = pd.read_csv('merge.csv')
df

,Date,Time,Lat,Long,Magnitude,Deaths,Tsunami
0,1901-03-03,07:45,36.2,−120.7,6.4,NaN,NaN
1,1901-03-31,07:10,NaN,NaN,7.2,4,NaN
2,1901-11-16,07:47,-42.42,173.18,6.9,NaN,NaN
3,1902-09-19,NaN,NaN,NaN,6.0,2,NaN
4,1902-04-19,02:23,14,−91,7.5,2000,NaN
...,...,...,...,...,...,...,...
2328529,2021-01-07,08:13:20,33.7563,-115.914,1.58,NaN,0.0
2328530,2021-01-07,08:13:20,35.9795,-97.3541,1.52,NaN,0.0
2328531,2021-01-07,08:13:20,59.9118,-151.122,1.6,NaN,0.0
2328532,2021-01-07,08:13:20,33.7503,-115.921,1.5,NaN,0.0


In [318]:
### Drop missing data: Date, Time, Lat and Long
### For being not useful
df = df.dropna(subset = ['Date', 'Time', 'Lat', 'Long'])
df

,Date,Time,Lat,Long,Magnitude,Deaths,Tsunami
0,1901-03-03,07:45,36.2,−120.7,6.4,NaN,NaN
2,1901-11-16,07:47,-42.42,173.18,6.9,NaN,NaN
4,1902-04-19,02:23,14,−91,7.5,2000,NaN
5,1902-12-16,05:04,40.8,72.3,6.4,4882,NaN
6,1903-04-29,01:46,39.14,42.65,6.7,3500,NaN
...,...,...,...,...,...,...,...
2328529,2021-01-07,08:13:20,33.7563,-115.914,1.58,NaN,0.0
2328530,2021-01-07,08:13:20,35.9795,-97.3541,1.52,NaN,0.0
2328531,2021-01-07,08:13:20,59.9118,-151.122,1.6,NaN,0.0
2328532,2021-01-07,08:13:20,33.7503,-115.921,1.5,NaN,0.0


In [319]:
df = df.sort_values(by=['Tsunami'], ascending = False).reset_index(drop=True)
df = df.drop_duplicates(subset =['Date', 'Time'],ignore_index=True)
df

,Date,Time,Lat,Long,Magnitude,Deaths,Tsunami
0,2016-08-01,00:20:00,2.0141,96.7462,5,NaN,1.0
1,2015-08-19,19:00:00,8.2309,-77.3152,5.9,NaN,1.0
2,2014-12-31,06:26:40,62.5442,-148.058,4.6,NaN,1.0
3,2018-06-26,11:00:00,-6.3924,146.253,5,NaN,1.0
4,2013-09-24,08:20:00,-30.2921,-71.5215,6.6,NaN,1.0
...,...,...,...,...,...,...,...
81827,2003-10-10,03:29:23,40.0,97.83333333333333,5.3,NaN,NaN
81828,2003-10-09,22:19:13,14.5,120.5,6.0,NaN,NaN
81829,2003-10-09,22:13:23,58.333333333333336,-32.0,5.4,NaN,NaN
81830,2003-10-09,16:06:07,50.333333333333336,88.16666666666667,5.1,NaN,NaN


In [320]:
### fix Magnitude data
df = df[~df.Magnitude.str.contains('–' ,na = False)]
df = df[~df.Magnitude.str.contains('-' ,na = False)]
df = df[~df.Magnitude.str.contains('&' ,na = False)]
df = df[~df.Magnitude.str.contains('\[' ,na = False)]
df

,Date,Time,Lat,Long,Magnitude,Deaths,Tsunami
0,2016-08-01,00:20:00,2.0141,96.7462,5,NaN,1.0
1,2015-08-19,19:00:00,8.2309,-77.3152,5.9,NaN,1.0
2,2014-12-31,06:26:40,62.5442,-148.058,4.6,NaN,1.0
3,2018-06-26,11:00:00,-6.3924,146.253,5,NaN,1.0
4,2013-09-24,08:20:00,-30.2921,-71.5215,6.6,NaN,1.0
...,...,...,...,...,...,...,...
81827,2003-10-10,03:29:23,40.0,97.83333333333333,5.3,NaN,NaN
81828,2003-10-09,22:19:13,14.5,120.5,6.0,NaN,NaN
81829,2003-10-09,22:13:23,58.333333333333336,-32.0,5.4,NaN,NaN
81830,2003-10-09,16:06:07,50.333333333333336,88.16666666666667,5.1,NaN,NaN


In [321]:
### convert the Magnitude data to numeric and remove all
### values less than 3 for being too frequent
df['Magnitude'] = df['Magnitude'].astype(float)
df = df[df['Magnitude']>3].reset_index(drop=True)
df

,Date,Time,Lat,Long,Magnitude,Deaths,Tsunami
0,2016-08-01,00:20:00,2.0141,96.7462,5.0,NaN,1.0
1,2015-08-19,19:00:00,8.2309,-77.3152,5.9,NaN,1.0
2,2014-12-31,06:26:40,62.5442,-148.058,4.6,NaN,1.0
3,2018-06-26,11:00:00,-6.3924,146.253,5.0,NaN,1.0
4,2013-09-24,08:20:00,-30.2921,-71.5215,6.6,NaN,1.0
...,...,...,...,...,...,...,...
80323,2003-10-10,03:29:23,40.0,97.83333333333333,5.3,NaN,NaN
80324,2003-10-09,22:19:13,14.5,120.5,6.0,NaN,NaN
80325,2003-10-09,22:13:23,58.333333333333336,-32.0,5.4,NaN,NaN
80326,2003-10-09,16:06:07,50.333333333333336,88.16666666666667,5.1,NaN,NaN


In [322]:
### fix Lat-Long data
df['Long'] = df['Long'].replace({'−': '-'},regex=True)
df['Lat'] = df['Lat'].replace({'−': '-'},regex=True)
df['Lat'] = df['Lat'].replace({':': '.'},regex=True)
df['Time'] = df['Time'].replace({'\.': ':'},regex=True)

df

,Date,Time,Lat,Long,Magnitude,Deaths,Tsunami
0,2016-08-01,00:20:00,2.0141,96.7462,5.0,NaN,1.0
1,2015-08-19,19:00:00,8.2309,-77.3152,5.9,NaN,1.0
2,2014-12-31,06:26:40,62.5442,-148.058,4.6,NaN,1.0
3,2018-06-26,11:00:00,-6.3924,146.253,5.0,NaN,1.0
4,2013-09-24,08:20:00,-30.2921,-71.5215,6.6,NaN,1.0
...,...,...,...,...,...,...,...
80323,2003-10-10,03:29:23,40.0,97.83333333333333,5.3,NaN,NaN
80324,2003-10-09,22:19:13,14.5,120.5,6.0,NaN,NaN
80325,2003-10-09,22:13:23,58.333333333333336,-32.0,5.4,NaN,NaN
80326,2003-10-09,16:06:07,50.333333333333336,88.16666666666667,5.1,NaN,NaN


In [323]:
### remove outliners
df['Long'] = df['Long'].astype('float')
df = df[df['Long']<=181].reset_index(drop=True)
df

,Date,Time,Lat,Long,Magnitude,Deaths,Tsunami
0,2016-08-01,00:20:00,2.0141,96.746200,5.0,NaN,1.0
1,2015-08-19,19:00:00,8.2309,-77.315200,5.9,NaN,1.0
2,2014-12-31,06:26:40,62.5442,-148.058100,4.6,NaN,1.0
3,2018-06-26,11:00:00,-6.3924,146.252700,5.0,NaN,1.0
4,2013-09-24,08:20:00,-30.2921,-71.521500,6.6,NaN,1.0
...,...,...,...,...,...,...,...
80321,2003-10-10,03:29:23,40.0,97.833333,5.3,NaN,NaN
80322,2003-10-09,22:19:13,14.5,120.500000,6.0,NaN,NaN
80323,2003-10-09,22:13:23,58.333333333333336,-32.000000,5.4,NaN,NaN
80324,2003-10-09,16:06:07,50.333333333333336,88.166667,5.1,NaN,NaN


In [324]:
### fix Deaths data
### i.e: `1,200` or `40-130` (not readable)
for i in range(len(df['Deaths'])):
    df['Deaths'][i] = (str(df['Deaths'][i]).replace(',', '')).replace('5.300','5300')
    if '-' in df['Deaths'][i]:
        df['Deaths'][i] = int(df['Deaths'][i].split('-')[0])+int(df['Deaths'][i].split('-')[1])/2
    elif '+' in df['Deaths'][i]:
        df['Deaths'][i] = int(df['Deaths'][i].split('+')[0])
    elif '[' in df['Deaths'][i]:
        df['Deaths'][i] = int(df['Deaths'][i].split('[')[0])
df

,Date,Time,Lat,Long,Magnitude,Deaths,Tsunami
0,2016-08-01,00:20:00,2.0141,96.746200,5.0,nan,1.0
1,2015-08-19,19:00:00,8.2309,-77.315200,5.9,nan,1.0
2,2014-12-31,06:26:40,62.5442,-148.058100,4.6,nan,1.0
3,2018-06-26,11:00:00,-6.3924,146.252700,5.0,nan,1.0
4,2013-09-24,08:20:00,-30.2921,-71.521500,6.6,nan,1.0
...,...,...,...,...,...,...,...
80321,2003-10-10,03:29:23,40.0,97.833333,5.3,nan,NaN
80322,2003-10-09,22:19:13,14.5,120.500000,6.0,nan,NaN
80323,2003-10-09,22:13:23,58.333333333333336,-32.000000,5.4,nan,NaN
80324,2003-10-09,16:06:07,50.333333333333336,88.166667,5.1,nan,NaN


In [325]:
df_tsunami = df[['Lat', 'Long', 'Magnitude', 'Tsunami']]
df_deaths = df[['Lat', 'Long', 'Magnitude', 'Deaths']]
df = df.drop(['Tsunami', 'Deaths'], axis = 1)
df_tsunami = df_tsunami.dropna().reset_index(drop=True)
df_deaths = df_deaths.replace('nan', float('nan')).dropna().reset_index(drop=True)

In [326]:
### Save relevant DF's
df_tsunami.to_csv('EDA\df_tsunami.csv',index=False)
df_deaths.to_csv('EDA\df_deaths.csv',index=False)
df.to_csv('EDA\df.csv',index=False)